In [33]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import operator
import matplotlib
matplotlib.use("Agg") #Needed to save figures
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

In [76]:
print("Load the training, test and store data using pandas")
types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(float),
         'PromoInterval': np.dtype(str)}
train = pd.read_csv("../data/train.csv", parse_dates=[2], dtype=types)
test = pd.read_csv("../data/test.csv", parse_dates=[3], dtype=types)
store = pd.read_csv("../data/store.csv")

train.fillna(1, inplace=True)
test.fillna(1, inplace=True)
train = train[train["Open"] != 0]
train = train[train["Sales"] > 0]

train = pd.merge(train, store, on='Store')

Load the training, test and store data using pandas


In [77]:
#     store['Sales25th'] = 0
#     store['Sales50th'] = 0
#     store['Sales75th'] = 0
#     store['SalesMedian'] = 0

#     for store_id in train.Store.unique():
#         Sales25th = train[train.Store == store_id].Sales.quantile(0.25)
#         Sales50th = train[train.Store == store_id].Sales.quantile(0.50)
#         Sales75th = train[train.Store == store_id].Sales.quantile(0.75)
#         SalesMedian = train[train.Store == store_id].Sales.median()

#         store.loc[store.Store == store_id, 'Sales25th'] = Sales25th
#         store.loc[store.Store == store_id, 'Sales50th'] = Sales50th
#         store.loc[store.Store == store_id, 'Sales75th'] = Sales75th
#         store.loc[store.Store == store_id, 'SalesMedian'] = SalesMedian

In [78]:
def build_features_store(store, train):
    train = train.copy()
#     train['Sales'] = train['Sales'].apply(np.log1p)
    train['Year'] = train.Date.dt.year
    train['Month'] = train.Date.dt.month
    train['Day'] = train.Date.dt.day
    train['DayOfWeek'] = train.Date.dt.dayofweek
    train['WeekOfYear'] = train.Date.dt.weekofyear
    
    store_ids = store.Store.unique()
    for store_id in store_ids:
        for day in train.DayOfWeek.unique():
            keyword = 'SalesDayOfWeek'
            name_median = keyword + str(day) + "_Median"
            name_25th = keyword + str(day) + "_25th"
            name_75th = keyword + str(day) + "_75th"
            store[name_median] = 0
            store[name_25th] = 0
            store[name_75th] = 0
#         for month in train.Month.unique():
#             keyword = 'SalesMonth'
#             name_median = keyword + str(month) + "_Median"
#             name_std = keyword + str(month) + "_std"
#             store[name_median] = 0
#             store[name_std] = 0
#         for year in train.Year.unique():
#             keyword = 'SalesYear'
#             name_median = keyword + str(year) + "_Median"
#             name_std = keyword + str(year) + "_std"
#             store[name_median] = 0
#             store[name_std] = 0
            
    
    for store_id in store_ids:
        for day in train.DayOfWeek.unique():
            keyword = 'SalesDayOfWeek'
            name_median = keyword + str(day) + "_Median"
            name_25th = keyword + str(day) + "_25th"
            name_75th = keyword + str(day) + "_75th"
            store.loc[store.Store == store_id, name_median] = train[(train.Store == store_id) & (train.DayOfWeek == day)].Sales.median()
            store.loc[store.Store == store_id, name_25th] = train[(train.Store == store_id) & (train.DayOfWeek == day)].Sales.quantile(0.25)
            store.loc[store.Store == store_id, name_75th] = train[(train.Store == store_id) & (train.DayOfWeek == day)].Sales.quantile(0.75)
#         for month in train.Month.unique():
#             keyword = 'SalesMonth'
#             name_median = keyword + str(month) + "_Median"
#             name_std = keyword + str(month) + "_std"
#             store.loc[store.Store == store_id, name_median] = train[(train.Store == store_id) & (train.Month == month)].Sales.median()
#             store.loc[store.Store == store_id, name_std] = train[(train.Store == store_id) & (train.Month == month)].Sales.std()
#         for year in train.Year.unique():
#             keyword = 'Salesyear'
#             name_median = keyword + str(year) + "_Median"
#             name_std = keyword + str(year) + "_stdth"
#             store.loc[store.Store == store_id, name_median] = train[(train.Store == store_id) & (train.Year == year)].Sales.median()
#             store.loc[store.Store == store_id, name_std] = train[(train.Store == store_id) & (train.Year == year)].Sales.std()
            
    store.fillna(0, inplace=True)        
    
build_features_store(store, train)
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,SalesDayOfWeek4_Median,SalesDayOfWeek4_25th,SalesDayOfWeek4_75th,SalesDayOfWeek3_Median,SalesDayOfWeek3_25th,SalesDayOfWeek3_75th,SalesDayOfWeek2_Median,SalesDayOfWeek2_25th,SalesDayOfWeek2_75th,SalesDayOfWeek1_Median,SalesDayOfWeek1_25th,SalesDayOfWeek1_75th,SalesDayOfWeek0_Median,SalesDayOfWeek0_25th,SalesDayOfWeek0_75th,SalesDayOfWeek5_Median,SalesDayOfWeek5_25th,SalesDayOfWeek5_75th,SalesDayOfWeek6_Median,SalesDayOfWeek6_25th,SalesDayOfWeek6_75th
0,1,c,a,1270,9,2008,0,0,0,0,4651,4025.00,5242.0,4380,3799.50,4964.50,4549.5,3843.50,5042.0,4640.5,3817.00,5401.50,5302.5,4067.00,5901.50,4785.0,4353.50,5314.5,0,0,0
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct",4671,4095.00,5016.0,5009,4111.00,5568.50,5920.5,4969.25,6617.0,5452.0,4077.75,6366.25,6530.5,4150.75,7369.00,2735.0,2553.50,3010.5,0,0,0
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct",7111,6178.00,8091.5,6940,5697.25,7909.25,7478.0,5512.50,8421.5,7928.0,5599.25,9059.75,8993.0,5604.00,10306.25,4221.0,3845.00,4725.0,0,0,0
3,4,c,c,620,9,2009,0,0,0,0,9353,8264.00,10416.0,8899,7812.50,9830.00,8902.0,7826.50,9610.0,9599.5,7653.50,10565.25,11106.5,8914.50,12170.50,9918.5,9331.25,10566.5,0,0,0
4,5,a,a,29910,4,2015,0,0,0,0,4792,4241.75,5228.5,4774,3870.50,5244.00,5308.5,4273.75,5964.0,5193.5,3846.50,6119.25,6846.0,4126.00,7615.00,2009.5,1805.50,2204.5,0,0,0


In [79]:
store.describe()

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,SalesDayOfWeek4_Median,SalesDayOfWeek4_25th,SalesDayOfWeek4_75th,SalesDayOfWeek3_Median,SalesDayOfWeek3_25th,SalesDayOfWeek3_75th,SalesDayOfWeek2_Median,SalesDayOfWeek2_25th,SalesDayOfWeek2_75th,SalesDayOfWeek1_Median,SalesDayOfWeek1_25th,SalesDayOfWeek1_75th,SalesDayOfWeek0_Median,SalesDayOfWeek0_25th,SalesDayOfWeek0_75th,SalesDayOfWeek5_Median,SalesDayOfWeek5_25th,SalesDayOfWeek5_75th,SalesDayOfWeek6_Median,SalesDayOfWeek6_25th,SalesDayOfWeek6_75th
count,1115.00000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.00000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.00000
mean,558.00000,5390.358744,4.930942,1370.939013,0.512108,12.083408,1030.239462,6964.233184,6115.139013,7729.555605,6625.654709,5694.526009,7472.76435,6656.327803,5548.923318,7608.771300,7120.430493,5582.124664,8236.889238,8558.810762,6005.742601,9868.423318,5733.591480,5282.783408,6272.739462,213.791031,190.827578,238.36278
std,322.01708,7657.973705,4.284924,935.467654,0.500078,15.542241,1006.038782,2343.723098,2135.123562,2542.501776,2287.693370,2035.908301,2503.50320,2304.230634,1999.813211,2556.875223,2482.552000,2035.353102,2772.535112,2880.457856,2181.509463,3233.738067,2605.053054,2439.451534,2814.232011,1588.737232,1453.861047,1727.53544
min,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2836.000000,1962.000000,3303.750000,2599.000000,2031.000000,2983.00000,2558.500000,1835.250000,2878.250000,2739.500000,1862.500000,3222.750000,3289.000000,1895.250000,3692.750000,866.500000,747.250000,1029.500000,0.000000,0.000000,0.00000
25%,279.50000,710.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5373.500000,4668.250000,6020.750000,5108.500000,4357.375000,5821.37500,5125.750000,4238.125000,5893.625000,5464.500000,4277.625000,6362.125000,6540.250000,4586.625000,7644.000000,3974.000000,3638.000000,4404.375000,0.000000,0.000000,0.00000
50%,558.00000,2320.000000,4.000000,2006.000000,1.000000,1.000000,2009.000000,6580.000000,5782.250000,7355.500000,6277.000000,5364.000000,7094.25000,6323.500000,5196.000000,7193.500000,6709.500000,5236.000000,7812.750000,8153.000000,5604.000000,9384.000000,5347.000000,4905.000000,5862.000000,0.000000,0.000000,0.00000
75%,836.50000,6875.000000,9.000000,2011.000000,1.000000,22.000000,2012.000000,7978.250000,7050.875000,8934.250000,7564.500000,6560.125000,8572.87500,7638.750000,6385.375000,8702.750000,8065.500000,6371.500000,9438.250000,9881.250000,6874.625000,11377.000000,7096.750000,6500.250000,7692.375000,0.000000,0.000000,0.00000
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000,21395.000000,19835.000000,22946.000000,22631.000000,20330.000000,24202.00000,22462.000000,20490.500000,24985.500000,23544.000000,21088.750000,25724.250000,24793.000000,22029.250000,28052.500000,21569.500000,20443.250000,22701.750000,28347.500000,26902.750000,30088.75000


In [80]:
store.to_csv("../data/store_features.pd", index=False)